In [47]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
import xgboost as xgb
import joblib

# data preprocessing

In [48]:
data_train = pd.read_csv("../data/for_train/train_without_teams_referee.csv")
data_train.head()
data_train.shape

(34613, 104)

In [49]:
data_train.drop(['Unnamed: 0','Total Goals','Match Kickoff_Not avaliable'], axis=1, inplace=True)

In [50]:
country_england = data_train[data_train['Country_england'] == 1]
country_france = data_train[data_train['Country_france'] == 1]

In [51]:
train_split = data_train[data_train['Season'] != 2122]
test_split = data_train[data_train['Season'] == 2122]
print(train_split.shape)
print(test_split.shape)

(27447, 101)
(7166, 101)


In [52]:
subsets = {category: subset for category, subset in data_train.groupby('League')}

for category, subset in subsets.items():
    globals()[f'dl{category}'] = subset

In [53]:
dl0.head()

,League,Season,FTR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,...,Country_germany,Country_greece,Country_italy,Country_netherlands,Country_portugal,Country_scotland,Country_spain,Country_turkey,Match Kickoff_afternoon,Match Kickoff_evening
1297,0,1718,2,1.53,4.5,6.50,1.50,4.60,6.75,1.47,...,False,False,False,False,False,False,False,False,False,False
1298,0,1718,0,11.00,5.5,1.33,11.00,5.25,1.30,8.00,...,False,False,False,False,False,False,False,False,False,False
1299,0,1718,0,1.83,3.6,5.00,1.80,3.50,4.75,1.85,...,False,False,False,False,False,False,False,False,False,False
1300,0,1718,0,1.25,6.5,15.00,1.22,6.50,12.50,1.22,...,False,False,False,False,False,False,False,False,False,False
1301,0,1718,2,1.70,3.8,5.75,1.70,3.60,5.50,1.70,...,False,False,False,False,False,False,False,False,False,False


In [54]:
label_encoder = LabelEncoder()

data_train['FTR'] = label_encoder.fit_transform(data_train['FTR'])

print(data_train['FTR'].value_counts())

2    14952
0    10542
1     9119
Name: FTR, dtype: int64


# models by league/country

In [55]:
scaler = StandardScaler()
def pipeline_season(model, df, season):
    X = df.drop('FTR', axis=1)
    y = df['FTR']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if season == True:
        print("Model: ",model, 'League: ',df['League'].iloc[0],"\n","Accuracy:",  metrics.accuracy_score(y_test, y_pred))
    else:
        print("Model: ",model,"\n","Accuracy:",  metrics.accuracy_score(y_test, y_pred), "F1 Score:", f1_score(y_test, y_pred, average='macro'))
    
def pipeline(model, train_split,test_split):
    X_train = train_split.drop('FTR', axis=1)
    X_test = test_split.drop('FTR', axis=1) 
    y_train = train_split['FTR']
    y_test = test_split['FTR']
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Model:",model,"\n", "Accuracy:", metrics.accuracy_score(y_test, y_pred), "F1 Score:", f1_score(y_test, y_pred, average='macro'))

Logistic Regression

In [56]:
model_rf = RandomForestClassifier()
pipeline_season(model_rf, dl0, True)
pipeline_season(model_rf, dl1, True)
pipeline_season(model_rf, dl2, True)
pipeline_season(model_rf, dl3, True)

Model:  RandomForestClassifier() League:  0 
 Accuracy: 0.5108077360637088
Model:  RandomForestClassifier() League:  1 
 Accuracy: 0.49819700132852535
Model:  RandomForestClassifier() League:  2 
 Accuracy: 0.44083023543990085
Model:  RandomForestClassifier() League:  3 
 Accuracy: 0.4405940594059406


In [57]:
pipeline_season(model_rf, country_england, False)
pipeline_season(model_rf, country_france, False)

Model:  RandomForestClassifier() 
 Accuracy: 0.4655231560891938 F1 Score: 0.39995490095324043
Model:  RandomForestClassifier() 
 Accuracy: 0.4664143803216651 F1 Score: 0.42486162404036837


Logistic regression

In [58]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs') 
pipeline_season(model_lr, dl0, True)
pipeline_season(model_lr, dl1, True)
pipeline_season(model_lr, dl2, True)
pipeline_season(model_lr, dl3, True)

/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model:  LogisticRegression(multi_class='multinomial') League:  0 
 Accuracy: 0.5472127417519909


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model:  LogisticRegression(multi_class='multinomial') League:  1 
 Accuracy: 0.5122414120326437
Model:  LogisticRegression(multi_class='multinomial') League:  2 
 Accuracy: 0.46065675340768275
Model:  LogisticRegression(multi_class='multinomial') League:  3 
 Accuracy: 0.43366336633663366


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

In [59]:
pipeline_season(model_lr, country_england, False)
pipeline_season(model_lr, country_france, False)

Model:  LogisticRegression(multi_class='multinomial') 
 Accuracy: 0.48336192109777015 F1 Score: 0.3741100413299476
Model:  LogisticRegression(multi_class='multinomial') 
 Accuracy: 0.4815515610217597 F1 Score: 0.4127554708812123


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

DecisionTree

In [60]:
model_DT = DecisionTreeClassifier()
pipeline_season(model_DT, dl0, True)
pipeline_season(model_DT, dl1, True)
pipeline_season(model_DT, dl2, True )
pipeline_season(model_DT, dl3, True)

Model:  DecisionTreeClassifier() League:  0 
 Accuracy: 0.43686006825938567
Model:  DecisionTreeClassifier() League:  1 
 Accuracy: 0.40994496109318657
Model:  DecisionTreeClassifier() League:  2 
 Accuracy: 0.3714374225526642
Model:  DecisionTreeClassifier() League:  3 
 Accuracy: 0.35346534653465345


In [61]:
pipeline_season(model_DT, country_england, False)
pipeline_season(model_DT, country_france, False)

Model:  DecisionTreeClassifier() 
 Accuracy: 0.37873070325900515 F1 Score: 0.3667424990971859
Model:  DecisionTreeClassifier() 
 Accuracy: 0.391674550614948 F1 Score: 0.37410984835775296


XGBClassifier

In [62]:
model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
pipeline_season(model_xgb, dl0, True)
pipeline_season(model_xgb, dl1, True)
pipeline_season(model_xgb, dl2, True)
pipeline_season(model_xgb, dl3, True)

Model:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...) League:  0 
 Accuracy: 0.4891922639362912
Model:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=Non

In [63]:
pipeline_season(model_xgb, country_england, False)
pipeline_season(model_xgb, country_france, False)

Model:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...) 
 Accuracy: 0.4542024013722127 F1 Score: 0.3897177445118459
Model:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytre

# main models

In [64]:
model1 = RandomForestClassifier()
model2 = LogisticRegression(multi_class='multinomial', solver='lbfgs') 
model3 = svm.SVC()
model4 = DecisionTreeClassifier()
model5 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [65]:
pipeline(model1, test_split, train_split)
pipeline(model2,  test_split, train_split)
pipeline(model3,  test_split, train_split)
pipeline(model4,  test_split, train_split)  
pipeline(model5,  test_split, train_split)

Model: RandomForestClassifier() 
 Accuracy: 0.3912631617298794 F1 Score: 0.3531047492998855


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: LogisticRegression(multi_class='multinomial') 
 Accuracy: 0.43709695048639197 F1 Score: 0.4016208010011568
Model: SVC() 
 Accuracy: 0.49258571064232887 F1 Score: 0.36728949792270066
Model: DecisionTreeClassifier() 
 Accuracy: 0.31376835355412247 F1 Score: 0.3129973666264033
Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=N

In [66]:
def tune_parameters(model, param_grid,  test_split, train_split, cv):
    X_train = train_split.drop('FTR', axis=1)
    X_test = test_split.drop('FTR', axis=1) 
    y_train = train_split['FTR']
    y_test = test_split['FTR']
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    clf= GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1_macro',
                        cv=cv, n_jobs=-1, verbose=2)
    clf.fit(X_train, y_train)
    best_parameters = clf.best_params_
    return best_parameters

In [67]:
param_grid_1 = {
            'n_estimators': [50, 100, 200, 300],
            'max_features': ['auto', 'sqrt', 'log2'],
            'max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8],
            'criterion': ['gini', 'entropy']
        }

param_grid_2 = {
    'C': np.logspace(-4, 4, 20),          
    'penalty': ['l2'],             
    'solver': ['lbfgs']               
}

param_grid_4 = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

param_grid_5 = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

In [68]:

best_parameters_model1 = tune_parameters(model=model1,param_grid=param_grid_1, test_split=test_split,train_split=  train_split, cv = 5)
best_parameters_model2 = tune_parameters(model=model2,param_grid=param_grid_2,test_split= test_split,train_split=train_split ,cv = 5)
best_parameters_model4 = tune_parameters(model=model4,param_grid=param_grid_4,
                                                test_split=test_split, train_split=train_split, cv = 5)
best_parameters_model5 = tune_parameters(model=model5,param_grid=param_grid_5, test_split=test_split,train_split=train_split, cv = 5)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_

[CV] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50; total time=  14.5s
[CV] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50; total time=  14.6s
[CV] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50; total time=  14.7s
[CV] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50; total time=  14.7s
[CV] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50; total time=  14.7s


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_

[CV] END criterion=gini, max_depth=None, max_features=auto, n_estimators=100; total time=  34.6s


...

[CV] END ................C=10000.0, penalty=l2, solver=lbfgs; total time=   2.0s


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

[CV] END ................C=10000.0, penalty=l2, solver=lbfgs; total time=   1.6s
[CV] END ................C=10000.0, penalty=l2, solver=lbfgs; total time=   1.6s
[CV] END ................C=10000.0, penalty=l2, solver=lbfgs; total time=   1.5s


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1

In [ ]:
print("Model:", model1,"Best Parameters:", best_parameters_model1)
print("Model:", model2,"Best Parameters:", best_parameters_model2)
print("Model:", model4,"Best Parameters:", best_parameters_model4)
print("Model:", model5,"Best Parameters:", best_parameters_model5)

Model: RandomForestClassifier() Best Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'n_estimators': 50}
Model: LogisticRegression(multi_class='multinomial') Best Parameters: {'C': 1438.44988828766, 'penalty': 'l2', 'solver': 'lbfgs'}
Model: DecisionTreeClassifier() Best Parameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}
Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=Non

In [ ]:
model1_tuned = RandomForestClassifier(max_depth=best_parameters_model1['max_depth'],
                                max_features=best_parameters_model1['max_features'],
                                n_estimators=best_parameters_model1['n_estimators'],
                                criterion=best_parameters_model1['criterion'])
model2_tuned = LogisticRegression(C=best_parameters_model2['C'],
                                penalty=best_parameters_model2['penalty'],
                                solver=best_parameters_model2['solver'])
model4_tuned = DecisionTreeClassifier(max_depth=best_parameters_model4['max_depth'], 
                                min_samples_split=best_parameters_model4['min_samples_split'], 
                                min_samples_leaf=best_parameters_model4['min_samples_leaf'], 
                                criterion=best_parameters_model4['criterion'])
model5_tuned = xgb.XGBClassifier(n_estimators=best_parameters_model5['n_estimators'],
                                learning_rate=best_parameters_model5['learning_rate'],
                                max_depth=best_parameters_model5['max_depth'],
                                subsample=best_parameters_model5['subsample'],
                                colsample_bytree=best_parameters_model5['colsample_bytree'],
                                use_label_encoder=False, eval_metric='mlogloss')

In [ ]:
pipeline(model1_tuned, test_split=test_split, train_split=train_split)
pipeline(model2_tuned, test_split=test_split, train_split=train_split)
pipeline(model4_tuned, test_split=test_split, train_split=train_split)
pipeline(model5_tuned, test_split=test_split, train_split=train_split)

/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Model: RandomForestClassifier(max_features='auto', n_estimators=50) 
 Accuracy: 0.47306726207089034 F1 Score: 0.42231617847579495


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: LogisticRegression(C=1438.44988828766) 
 Accuracy: 0.502511861568518 F1 Score: 0.3906416214661264
Model: DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=4) 
 Accuracy: 0.5043259838124476 F1 Score: 0.4131072224006767
Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, objective

In [ ]:
#joblib.dump(model1_tuned, 'model1_tuned.joblib')
#joblib.dump(model2_tuned, 'model2_tuned.joblib')
#joblib.dump(model4_tuned, 'model4_tuned.joblib')
#joblib.dump(model5_tuned, 'model5_tuned.joblib')

In [ ]:
X = data_train.drop('FTR', axis=1)
importances = model1_tuned.feature_importances_
feature_names = X.columns
feature_importances = dict(zip(feature_names, importances))
sorted_feature_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)
for feature, importance in sorted_feature_importances:
    print(f"Feature: {feature}, Importance: {importance}")

Feature: PSCA, Importance: 0.04356395014201524
Feature: PSCH, Importance: 0.03989823231799167
Feature: date_int, Importance: 0.036450738585597345
Feature: PSCD, Importance: 0.03434743459598174
Feature: PSD, Importance: 0.03323908553590629
Feature: PSA, Importance: 0.032098513039715934
Feature: PSH, Importance: 0.03118970164104292
Feature: IWA, Importance: 0.02975022838063803
Feature: IWH, Importance: 0.026773343942255817
Feature: B365H, Importance: 0.02534192044559749
Feature: BWH, Importance: 0.025022982566547485
Feature: BWA, Importance: 0.025021927400632322
Feature: VCA, Importance: 0.023832236361077147
Feature: B365A, Importance: 0.02337973103882384
Feature: IWD, Importance: 0.0228556111638253
Feature: WHA, Importance: 0.022743231139336784
Feature: VCH, Importance: 0.021693920148212316
Feature: WHH, Importance: 0.021515012005837615
Feature: BWD, Importance: 0.01989266342946875
Feature: B365D, Importance: 0.019616031897993037
Feature: VCD, Importance: 0.019143260659111812
Feature: W

In [ ]:
test_df = pd.read_csv("../data/test/test_final_no_teams_no_referee.csv")
test_df.drop(['Unnamed: 0'], axis=1, inplace=True)


In [ ]:
print(len(data_train.columns), len(X.columns) , len(test_df.columns))

print(X.head())

102 101 100
   League  Season  B365H  B365D  B365A   BWH  BWD   BWA   IWH  IWD  ...  \
0       1    1718   5.75   3.80   1.60  5.00  4.0  1.67  4.70  3.6  ...   
1       1    1718   2.60   3.40   2.60  2.70  3.4  2.60  2.65  3.3  ...   
2       1    1718   1.62   3.75   5.50  1.67  3.7  5.50  1.60  3.7  ...   
3       1    1718   1.40   4.75   7.00  1.42  4.6  7.75  1.40  4.4  ...   
4       1    1718   4.20   3.60   1.83  4.25  3.7  1.83  4.20  3.5  ...   

   Country_greece  Country_italy  Country_netherlands  Country_portugal  \
0           False          False                False             False   
1           False          False                False             False   
2           False          False                False             False   
3           False          False                False             False   
4           False          False                False             False   

   Country_scotland  Country_spain  Country_turkey  \
0             False          Fal

In [ ]:
test_df['Predicted_Target'] = model1_tuned.predict(test_df)

/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 100 features, but RandomForestClassifier is expecting 101 features as input.

In [ ]:
countries = ["belgium", "england", "france", "germany", "greece", "italy", "netherlands", "portugal", "scotland", "spain", "turkey"]
for league in test_df['League'].unique():
    league_df = test_df[test_df['League'] == league]
    for country in countries:
        country_col = f'Country_{country}'
        if country_col in test_df.columns:
            country_subset = league_df[league_df[country_col] == 1]
            if not country_subset.empty:
                country_subset['Predicted_Target'].to_csv(f'{country}_{league}.csv', index=False, header=False)